In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-06 23:51:41--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-06 23:51:41 (17.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [2]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [3]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [4]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [8]:
llm('write that this is a test')

' This is a test.\n\n\nTo execute the command "write that this is a test," I would input or speak:\n\n"This is a test." It\'s straightforward, as it simply involves stating what has been written previously by oneself—confirmation of content creation without additional embellishments.'

In [9]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/02-open-source/rag-intro.ipynb


--2024-07-07 00:19:21--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/02-open-source/rag-intro.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7380 (7.2K) [text/plain]
Saving to: ‘rag-intro.ipynb’

rag-intro.ipynb     100%[===================>]   7.21K  --.-KB/s    in 0.001s  

2024-07-07 00:19:21 (10.7 MB/s) - ‘rag-intro.ipynb’ saved [7380/7380]



In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/02-open-source/qa_faq.py


--2024-07-07 11:28:34--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/02-open-source/qa_faq.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2271 (2.2K) [text/plain]
Saving to: ‘qa_faq.py’

qa_faq.py           100%[===================>]   2.22K  --.-KB/s    in 0s      

2024-07-07 11:28:34 (28.6 MB/s) - ‘qa_faq.py’ saved [2271/2271]

